<a href="https://colab.research.google.com/github/nevemarpole/ColabCode/blob/main/BERT_just_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

RuntimeError: ignored

In [ ]:
!pip install pytorch-pretrained-bert pytorch-nlp

     |████████████████████████████████| 133kB 18.6MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 133kB 26.3MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 7.3MB 51.1MB/s 
ERROR: botocore 1.20.39 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
df = pd.read_csv("train.csv", delimiter=',', usecols=('conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance'))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.shape

(79226, 6)

In [ ]:
df.sample(10)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance
35551,hit:5536_conv:11073,2,grateful,I am grateful that I am able to stay home with...,10,I am so thankful that I am able to stay home w...
14175,hit:2203_conv:4406,3,disappointed,I was pretty let down when I didn't get a good...,267,I am working harder to get another job
1780,hit:260_conv:521,1,guilty,I stayed up later than I had planned last nigh...,85,I had way too many beers last night_comma_ and...
46947,hit:7162_conv:14325,4,sad,My girlfriend dumped me yesterday. I am so hu...,516,Was she honest about it or did you find out th...
46751,hit:7141_conv:14283,4,grateful,My sister helped me to study for my finals and...,408,Was is a general eduaction class or one toward...
66984,hit:10415_conv:20831,2,confident,I changed out my car's thermostat the other da...,706,it's insane how Youtube and the internet overa...
45085,hit:6920_conv:13841,4,joyful,I felt this way when I held my son for the fir...,1,Cherish him while you can. They really grow up...
15065,hit:2320_conv:4640,2,faithful,There's a new hotel that's been headhunting me...,384,I'd probably stay loyal too_comma_ after weigh...
37090,hit:5745_conv:11491,2,anxious,waiting for a package to arrive,196,Wow! I don't know much about baseball_comma_ b...
20266,hit:3109_conv:6219,1,lonely,I went hiking all alone yesterday because my f...,209,I went hiking all by my lonesome yesterday bec...


In [ ]:
# Create sentence and label lists
sentences = df.prompt.values

i = 0
for this in sentences:
    sentences[i] = str(sentences[i])
    i = i + 1

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]

df['context'].replace({"surprised": "0", "excited": "1", "angry": "2", "proud": "3", 
                         "sad": "4", "annoyed": "5", "grateful": "6", "lonely": "7", 
                         "afraid": "8", "terrified": "9", "guilty": "10", "impressed": "11",
                         "disgusted": "12", "hopeful": "13", "confident": "14", 
                         "furious": "15", "anxious": "16", "anticipating": "17",
                         "joyful": "18", "nostalgic": "19", "disappointed": "20",
                         "prepared": "21", "jealous": "22", "content": "23",
                         "devastated": "24", "embarrassed": "25", "caring": "26",
                         "sentimental": "27", "trusting": "28", "ashamed": "29",
                         "apprehensive": "30", "faithful": "31",}, inplace=True)

labels = df.context.values

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 231508/231508 [00:00<00:00, 17533041.97B/s]


Tokenize the first sentence:
['[CLS]', 'i', 'remember', 'going', 'to', 'the', 'fireworks', 'with', 'my', 'best', 'friend', '.', 'there', 'was', 'a', 'lot', 'of', 'people', '_', 'com', '##ma', '_', 'but', 'it', 'only', 'felt', 'like', 'us', 'in', 'the', 'world', '.', '[SEP]']


In [ ]:
MAX_LEN = 128

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [ ]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
# Convert all of our data into torch tensors, the required datatype for our model
print(train_inputs)

train_labels = np.array(train_labels, dtype='float32')
validation_labels = np.array(validation_labels, dtype='float32')

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

[[ 101 1045 2215 ...    0    0    0]
 [ 101 1045 3685 ...    0    0    0]
 [ 101 1045 2165 ...    0    0    0]
 ...
 [ 101 1045 2031 ...    0    0    0]
 [ 101 2026 4845 ...    0    0    0]
 [ 101 3403 2005 ...    0    0    0]]


In [ ]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=32)
model.cuda()

RuntimeError: ignored

In [ ]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [1]:
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device, dtype=torch.int64) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

NameError: ignored